## รับข้อมูลการผลิตไฟจากInverter

In [58]:
import numpy as np
import pandas as pd

uri = input("Enter data name to load (without extension): ")

if not uri.endswith('.xlsx'):
    uri += '.xlsx'

df = pd.read_excel(uri)
# โหลดไฟล์ใหม่จากผู้ใช้อัปโหลดล่าสุด

df.columns.name = None

# ค้นหาแถวที่มีชื่อคอลัมน์จริง (แถวที่มี "Start Time")
header_row_index = df[df.eq("Start Time").any(axis=1)].index[0]

# ตั้งค่าหัวตารางใหม่
df.columns = df.iloc[header_row_index]
df = df.iloc[header_row_index + 1:].reset_index(drop=True)

# เปลี่ยนชื่อคอลัมน์ให้เป็นมาตรฐาน
df.rename(columns={"Start Time": "Time"}, inplace=True)

# ตรวจสอบค่าดิบของคอลัมน์ Time ก่อนแปลง
df["Time"] = df["Time"].astype(str)  # แปลงให้เป็น string ก่อนเพื่อตรวจสอบค่า

# แสดงค่าเริ่มต้นของคอลัมน์ Time
df_time_sample = df[["Time"]].drop_duplicates().head(20)

# แปลงคอลัมน์เวลาเป็น datetime พร้อมแสดงเวลา
df["Time"] = pd.to_datetime(df["Time"], errors='coerce')

# ละทิ้งคอลัมน์ที่ไม่จำเป็น (ข้อมูลที่ไม่เกี่ยวข้องกับ PV readings)
df_filtered_new = df.drop(columns=["Site Name", "Management Domain", "ManageObject"], errors='ignore')

# Melt ข้อมูลเพื่อให้มีคอลัมน์ Parameter และ Value
df_melted_new = df_filtered_new.melt(id_vars=["Time"], var_name="Parameter", value_name="Value")

# แยกค่า PV_no และ ประเภทของพารามิเตอร์ (กระแส/แรงดัน)
df_melted_new["PV_no."] = df_melted_new["Parameter"].str.extract(r'(\d+)').astype("Int64")
df_melted_new["Parameter"] = df_melted_new["Parameter"].str.replace(r'\d+', '', regex=True).str.strip()

# Pivot Table ให้เป็นรูปแบบที่ต้องการ
df_final_new = df_melted_new.pivot_table(index=["Time", "PV_no."], columns="Parameter", values="Value").reset_index()

# จัดเรียงคอลัมน์ให้ตรงกับตัวอย่าง
df = df_final_new.rename(columns={"input current(A)": "PV_input_current(A)", 
                                            "input voltage(V)": "PV_input_voltage(V)"})

df.head(60)



c:\Users\Acer\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Parameter,Time,PV_no.,PV input current(A),PV input voltage(V)
0,2024-11-01 00:00:00,1,0.0,0.0
1,2024-11-01 00:00:00,2,0.0,0.0
2,2024-11-01 00:00:00,3,0.0,0.0
3,2024-11-01 00:00:00,4,0.0,0.0
4,2024-11-01 00:00:00,5,0.0,0.0
5,2024-11-01 00:00:00,6,0.0,0.0
6,2024-11-01 00:00:00,7,0.0,0.0
7,2024-11-01 00:00:00,8,0.0,0.0
8,2024-11-01 00:00:00,9,0.0,0.0
9,2024-11-01 00:00:00,10,0.0,0.0


## รับข้อมูลการวัดสภาพแวดล้อมจาก EMI

In [59]:
import numpy as np
import pandas as pd
emi_uri = input("Enter data name (without extension): ")

if not emi_uri.endswith('.xlsx'):
    emi_uri += '.xlsx'

df_emi = pd.read_excel(emi_uri)
df_emi.drop(df_emi.columns[[0,1,2]],axis=1,inplace=True)
df_emi.drop([0,1,2,3],axis=0,inplace=True)
df_emi.columns = ['Time','Ambient_temp','Irradiance','PV_temp']
df_emi['Time'] = pd.to_datetime(df_emi['Time'])
df_emi.set_index(['Time'],inplace=True)
df_emi


c:\Users\Acer\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Ambient_temp,Irradiance,PV_temp
Time,,,
2024-11-01 00:15:00,28.9,0.9,26.7
2024-11-01 00:30:00,28.7,0.9,26.8
2024-11-01 00:45:00,28.7,0.9,26.7
2024-11-01 01:00:00,28.6,0.3,26.8
2024-11-01 01:15:00,28.5,0.9,26.5
...,...,...,...
2024-11-30 22:45:00,0,0,23.3
2024-11-30 23:00:00,0,0,23
2024-11-30 23:15:00,0,0,22.9


## รวมข้อมูลผลิตกระแสและสภาพแวดล้อม

In [60]:
merged_df = pd.merge(df,df_emi,  on='Time', how='left')
merged_df.fillna(0, inplace=True)
data = merged_df
data = data[['Time','PV input current(A)','PV input voltage(V)','Ambient_temp','Irradiance','PV_temp','PV_no.']]
data

C:\Users\Acer\AppData\Local\Temp\ipykernel_16848\572529306.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df.fillna(0, inplace=True)


,Time,PV input current(A),PV input voltage(V),Ambient_temp,Irradiance,PV_temp,PV_no.
0,2024-11-01 00:00:00,0.0,0.0,0.0,0.0,0.0,1
1,2024-11-01 00:00:00,0.0,0.0,0.0,0.0,0.0,2
2,2024-11-01 00:00:00,0.0,0.0,0.0,0.0,0.0,3
3,2024-11-01 00:00:00,0.0,0.0,0.0,0.0,0.0,4
4,2024-11-01 00:00:00,0.0,0.0,0.0,0.0,0.0,5
...,...,...,...,...,...,...,...
9475,2024-11-19 18:15:00,0.0,205.4,31.2,0.3,30.1,6
9476,2024-11-19 18:15:00,0.0,205.4,31.2,0.3,30.1,7
9477,2024-11-19 18:15:00,0.0,205.4,31.2,0.3,30.1,8
9478,2024-11-19 18:15:00,0.0,205.2,31.2,0.3,30.1,9


In [61]:
# แก้ไขชื่อคอลัมน์ให้ตรงกับที่กำหนด
column_mapping = {
    "Time": "Time",
    "PV input current(A)": "PV_input_current(A)",
    "PV input voltage(V)": "PV_input_voltage(V)",
    "Ambient Temperature": "Ambient_temp",
    "Solar Irradiance": "Irradiance",
    "PV Panel Temperature": "PV_temp",
    "PV_no": "PV_no.",
    "Anomaly Detection": "Anomaly_Label"
}

# ใช้ mapping เปลี่ยนชื่อคอลัมน์
data.rename(columns=column_mapping, inplace=True)

data

,Time,PV_input_current(A),PV_input_voltage(V),Ambient_temp,Irradiance,PV_temp,PV_no.
0,2024-11-01 00:00:00,0.0,0.0,0.0,0.0,0.0,1
1,2024-11-01 00:00:00,0.0,0.0,0.0,0.0,0.0,2
2,2024-11-01 00:00:00,0.0,0.0,0.0,0.0,0.0,3
3,2024-11-01 00:00:00,0.0,0.0,0.0,0.0,0.0,4
4,2024-11-01 00:00:00,0.0,0.0,0.0,0.0,0.0,5
...,...,...,...,...,...,...,...
9475,2024-11-19 18:15:00,0.0,205.4,31.2,0.3,30.1,6
9476,2024-11-19 18:15:00,0.0,205.4,31.2,0.3,30.1,7
9477,2024-11-19 18:15:00,0.0,205.4,31.2,0.3,30.1,8
9478,2024-11-19 18:15:00,0.0,205.2,31.2,0.3,30.1,9


## บันทึก csv และ excel

In [64]:
file_uri = input("Enter data name to load (without extension): ")

data.to_csv(f"{file_uri}.csv")
data.to_excel(f"{file_uri}.xlsx")

In [66]:
df = data[data['Irradiance'] > 0].reset_index(drop=True)
df

,Time,PV_input_current(A),PV_input_voltage(V),Ambient_temp,Irradiance,PV_temp,PV_no.
0,2024-11-01 06:15:00,0.01,511.7,27.0,0.9,24.7,1
1,2024-11-01 06:15:00,0.03,511.7,27.0,0.9,24.7,2
2,2024-11-01 06:15:00,0.0,562.6,27.0,0.9,24.7,3
3,2024-11-01 06:15:00,0.04,562.6,27.0,0.9,24.7,4
4,2024-11-01 06:15:00,0.0,551.3,27.0,0.9,24.7,5
...,...,...,...,...,...,...,...
9465,2024-11-19 18:15:00,0.0,205.4,31.2,0.3,30.1,6
9466,2024-11-19 18:15:00,0.0,205.4,31.2,0.3,30.1,7
9467,2024-11-19 18:15:00,0.0,205.4,31.2,0.3,30.1,8
9468,2024-11-19 18:15:00,0.0,205.2,31.2,0.3,30.1,9
